In [234]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
import mapclassify
import matplotlib.pyplot as plt
from matplotlib import colors
from mapclassify import classify
from folium.plugins import MousePosition
from folium.plugins import Fullscreen
from folium.plugins import AntPath
from folium.features import GeoJsonTooltip
from folium.features import CustomIcon
from folium.plugins import Search



from shapely import geometry

In [235]:
data = gpd.read_file('./hotels_points.geojson')
data_cafe = gpd.read_file('./cafe.geojson')
data_poly = gpd.read_file('./border_.geojson')
data_poly_1 = gpd.read_file('./okn_polygon.geojson')
data_okn_points = gpd.read_file('./okn_points.geojson')
data_route = gpd.read_file('./route.geojson')
data.head()

,fid,name,price,price_int,geometry
0,1,Кремлин Лофт,от 34 856 руб,34856,POINT (589102.627 6253521.67)
1,2,Генеральская дача,от 19 937 руб,19937,POINT (589198.061 6253409.501)
2,3,Онегин,от 12 500 руб,12500,POINT (589254.823 6253376.258)
3,4,Флигель доктора Моренкова,от 7 169 руб,7169,POINT (589143.107 6253341.884)
4,5,Гостевой дом Терем Гамаюн,от149 033 руб,149033,POINT (589057.85 6253221.574)


In [236]:
print(data.crs.name)

WGS 72BE / UTM zone 37N


In [237]:
total_bounds = data.total_bounds
minX, minY, maxX, maxY = total_bounds

In [238]:
square_size = 300

In [239]:
grid_cells = []
x, y = (minX, minY)
geom_array = []

while y <= maxY:
        while x <= maxX:
            geom = geometry.Polygon([(x,y), (x, y+square_size), (x+square_size, y+square_size), (x+square_size, y), (x, y)])
            geom_array.append(geom)
            x += square_size
        x = minX
        y += square_size


fishnet = gpd.GeoDataFrame(geom_array, columns=['geometry']).set_crs('EPSG:32437')
fishnet['id'] = fishnet.index

In [240]:
# Присоединяем точки отелей к ячейкам сетки
joined_data = gpd.sjoin(fishnet, data, how='left', predicate='contains')

# Вычисляем среднюю цену для каждой ячейки и записываем её в fishnet
fishnet['avg_price'] = joined_data.groupby('id')['price_int'].mean().values


In [241]:
grid_cells_1 = []
x, y = (minX, minY)
geom_array = []

while y <= maxY:
        while x <= maxX:
            geom = geometry.Polygon([(x,y), (x, y+square_size), (x+square_size, y+square_size), (x+square_size, y), (x, y)])
            geom_array.append(geom)
            x += square_size
        x = minX
        y += square_size


fishnet_1 = gpd.GeoDataFrame(geom_array, columns=['geometry']).set_crs('EPSG:32437')
fishnet_1['id'] = fishnet_1.index

In [242]:
merged = gpd.sjoin(data_okn_points, fishnet_1, how='left', predicate='within')
merged['n'] = 1
dissolve = merged.dissolve(by="index_right", aggfunc="count")
fishnet_1.loc[dissolve.index, 'n'] = dissolve.n.values

In [243]:
# Создаем карту
data = data.to_crs('EPSG:4326')
m = folium.Map(location=[data.centroid.y.mean(), data.centroid.x.mean()], zoom_start=12,  tiles="cartodb positron", control_scale=True)


C:\Users\User\AppData\Local\Temp\ipykernel_13008\1711575688.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[data.centroid.y.mean(), data.centroid.x.mean()], zoom_start=12,  tiles="cartodb positron", control_scale=True)


In [244]:
# Создаем heatmap достопримечательностей
folium.Choropleth(
    geo_data=fishnet_1, 
    data=fishnet_1,                      
    columns=['id', 'n'],               
    key_on='feature.id',               
    fill_color='YlOrRd',                 
    fill_opacity=0.7,
    nan_fill_opacity=0,
    line_opacity=0.2,
    line_color='white',
    legend_name='Количество достопримечательностей',
    name='Концентрация достопримечательностей',
).add_to(m)


In [245]:
# Создаем картограмму средней стоимости проживания за ночь
folium.Choropleth(
    geo_data=fishnet,
    data=fishnet,
    columns=['id', 'avg_price'],
    fill_color='YlGnBu',
    fill_opacity = 0.5,
    key_on='id',
    nan_fill_opacity=0,
   line_color = "#0000",
   legend_name="Средняя цена на проживание за ночь (руб)",
   name='Распределение средней цены на проживание'
).add_to(m)


In [246]:
# Добавляем слой с границами
folium.GeoJson(
    data_poly,
    name='Границы г. Суздаль',
    style_function=lambda x: {
        'color': 'red',
        'weight': 2,
        'opacity': 1,
        'fillOpacity': 0
    }
).add_to(m)

In [247]:
# Добавляем слой с пешеходным маршрутом
folium.GeoJson(
    data_route,
    name='Предполагаемый туристический пешеходный маршрут',
    style_function=lambda x: {
        'color': 'purple',
        'weight': 1,
        'dashArray': '5,5',
        'opacity': 1,
        'fillOpacity': 0
    }
).add_to(m)

In [248]:
# Полигоны ключевых достопримечательностей
folium.GeoJson(
    data_poly_1,
    name='Достопримечательности',
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'darkblue',
        'weight': 1,
        'fillOpacity': 0.3,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['name'],
        aliases=['Достопримечательность: '],
        sticky=True,
        style="""
            background-color: white;
            border: 1px solid black;
            border-radius: 3px;
            padding: 5px;
            font-family: Arial;
            font-size: 14px;
        """
    ),
    # Добавляем зум при клике
    highlight_function=lambda x: {'weight': 3, 'color': 'red'}, 
    zoom_on_click=True  
).add_to(m)

In [249]:
# Точки отелей + возможность поиска по названию
hotels_group = folium.FeatureGroup(name='Отели')

folium.GeoJson(
    data,  
    name='Отели',
    marker=folium.Marker(
        icon=folium.Icon(
            icon='hotel',  
            prefix='fa',
            color='blue'
        )
    ),
    tooltip=GeoJsonTooltip(
        fields=['name', 'price'],
        aliases=['Отель:', 'Цена за ночь:'],
        localize=True,
        sticky=True,
        style="""
            background-color: #F0F8FF;
            border: 2px solid #4682B4;
            border-radius: 5px;
            font-family: Arial;
            padding: 5px;
        """
    )
).add_to(hotels_group)

hotels_group.add_to(m)

Search(
    layer=hotels_group,
    search_label='name',  
    placeholder='Найти отель...',
    collapsed=False,
    position='topleft',
    search_zoom=17,  
    zoom_on_click=True,
    style="""
        width: 300px;
        font-size: 14px;
    """
).add_to(m)

In [250]:
# Точки кафе + возможность поиска кафе по названию
restaurants_group = folium.FeatureGroup(name='Кафе')

folium.GeoJson(
    data_cafe,
    name='Кафе',
    marker=folium.Marker(
        icon=folium.Icon(icon='mug-hot', prefix='fa')
    ),
    tooltip=GeoJsonTooltip(
        fields=['name', 'price'],
        aliases=['Кафе:', 'Средний чек:'],
        localize=True,
        sticky=True
    )
).add_to(restaurants_group)

restaurants_group.add_to(m)

Search(
    layer=restaurants_group,
    search_label='name',  
    placeholder='Поиск кафе по названию',
    collapsed=False,
    position='topleft'
).add_to(m)

In [251]:
# Добавляем MousePosition
MousePosition().add_to(m)
Fullscreen(
    position="bottomright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)


In [ ]:
# Добавляем управление слоями
folium.LayerControl().add_to(m)
m

In [ ]:
m.save("index.html")